In [ ]:
import os

spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.8 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [ ]:
## import postgress SQL 
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar


--2022-03-06 17:46:01--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.71MB/s    in 0.2s    

2022-03-06 17:46:02 (5.71 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
## Create Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


In [ ]:
from pyspark import SparkFiles
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
autos_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Automotive_v1_00.tsv.gz"), sep="\t", header=True)
autos_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [ ]:
### Drop the null values

autos_df = autos_df.dropna()

autos_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
### SQL SChema  review_id, customer_id, product_id,  product_parent, review_date make the SQL schema rows into integers

# giftcard_df =giftcard_df.withColumn(("customer_id", giftcard_df["customer_id"].cast(IntegerType()))
# giftcard_df = giftcard_df.withColumn(("star_rating", giftcard_df["star_rating"].cast(IntegerType()))
# giftcard_df = giftcard_df.withColumn("helpful_votes", giftcard_df["helpful_votes"].cast(IntegerType()))
# giftcard_df = giftcard_df.withColumn("total_votes", giftcard_df["total_votes"].cast(IntegerType()))
# giftcard_df = giftcard_df.withColumn(("product_parent", giftcard_df["product_parent"].cast(IntegerType()))
autos_df = autos_df.withColumn("customer_id", autos_df["customer_id"].cast(IntegerType()))
autos_df = autos_df.withColumn("product_parent", autos_df["product_parent"].cast(IntegerType()))
autos_df = autos_df.withColumn("star_rating", autos_df["star_rating"].cast(IntegerType()))
autos_df= autos_df.withColumn("helpful_votes", autos_df["helpful_votes"].cast(IntegerType()))
autos_df = autos_df.withColumn("total_votes", autos_df["total_votes"].cast(IntegerType()))

In [ ]:
## Transform data to SQL and change
from pyspark.sql.functions import to_date

review_id_table= autos_df.select(["review_id",  "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RAB23OVFNCXZQ|   36075342|B00LPRXQ4Y|     339193102| 2015-08-31|
|R3NORADVJO6IE6|   42462164|B000C7S0TO|     907684644| 2015-08-31|
|R299F4SO98S5OO|   21241933|B000CO9WE4|     752246352| 2015-08-31|
|R2DA9DOT03UW6I|   52570308|B000GKD5NI|     105401756| 2015-08-31|
|R2OGCH681EQHU6|   38200102|B009SDA7TE|     728471129| 2015-08-31|
| R2JMKIC16MHD7|   34866169|B00KV15KRG|     962286893| 2015-08-31|
|R1DB5DA7CWWTI8|     184627|B0002JMAKW|     267002949| 2015-08-31|
|R1N8XWFDK4QACP|    2975964|B00XJKMM6S|     570789093| 2015-08-31|
|R19VFLX6MRU0CX|   40946484|B000C5CEKC|     389524802| 2015-08-31|
| RLTD8YDGTP9PT|   12554469|B00GD9R2A8|      21425394| 2015-08-31|
| R78B5LCFJ52SR|   35335277|B00RM327NG|     816815445| 2015-08-31|
|R33SKWL0HEQIQ9|   44957003|B0071NL7UQ|     192225755| 2015-08

In [ ]:
review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
## Build products Schema
products=autos_df.select(["product_id", "product_title"])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00LPRXQ4Y|17" 2003-2006 For...|
|B000C7S0TO|Spectra Premium C...|
|B000CO9WE4|K&N E-4665 High P...|
|B000GKD5NI|Suncutters Rear W...|
|B009SDA7TE|Lug Nuts Landcrui...|
|B00KV15KRG|Fits 2007-2009 Ho...|
|B0002JMAKW|Castrol 12614 Dot...|
|B00XJKMM6S|New Power Window ...|
|B000C5CEKC|Motorad MGC-791 L...|
|B00GD9R2A8|Genuine Ford AL3Z...|
|B00RM327NG|Air Vent Smartpho...|
|B0071NL7UQ|FDJ New Replaceme...|
|B00DSOL0FY|Omix-Ada 17402.09...|
|B00GRV48TK|GMB 530-2480 Fuel...|
|B000E4PCGE|00-03 NISSAN SENT...|
|B005Z8HAUK|Slime 40026 2X He...|
|B0043AV4NK|MegaBrand N1 4" B...|
|B00CDHD76C|Jackly 45-in-One ...|
|B000CF1XO0|Bosch 3397118942 ...|
|B003VLBPJA|Heavy-Duty 15" Ca...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
### Build customer table

##create customer total count
customers = autos_df.groupby(["customer_id"]).count()
customers = customers.withColumnRenamed("count", "customer_count")
customers.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [ ]:
##### Build the vine table 
vine_table= autos_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RAB23OVFNCXZQ|          1|            0|          0|   N|
|R3NORADVJO6IE6|          5|            0|          0|   N|
|R299F4SO98S5OO|          5|            1|          1|   N|
|R2DA9DOT03UW6I|          5|            2|          3|   N|
|R2OGCH681EQHU6|          5|            0|          0|   N|
| R2JMKIC16MHD7|          5|            2|          2|   N|
|R1DB5DA7CWWTI8|          5|            0|          0|   N|
|R1N8XWFDK4QACP|          5|            0|          0|   N|
|R19VFLX6MRU0CX|          5|            0|          0|   N|
| RLTD8YDGTP9PT|          1|            0|          0|   N|
| R78B5LCFJ52SR|          4|            0|          0|   N|
|R33SKWL0HEQIQ9|          5|            0|          0|   N|
|R1HIOV0UX7P13D|          5|            0|          0|   N|
| R4WS0E0MSP9DH|          5|            

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<connection string>:5432/bigdata"
config = {"user":"root", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

## Ran it and changed my password pack

In [ ]:
### Write Dataframe for the 4x SQL Schemas 

review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
vine_table.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)